You can download the dataset from: https://www.kaggle.com/datasets/msambare/fer2013

In [2]:
training_path = '/kaggle/input/fer2013/train'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-07-17 11:37:03.229302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 11:37:03.229405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 11:37:03.369121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
image_generator = ImageDataGenerator(validation_split = 0.2)

train_batchs = image_generator.flow_from_directory(
    directory = training_path,
    target_size = (96, 96),
    batch_size = 32,
    subset = 'training',
    class_mode = 'categorical'
)

validation_batchs = image_generator.flow_from_directory(
    directory = training_path,
    target_size = (96, 96),
    batch_size = 32,
    subset = 'validation',
    class_mode = 'categorical'
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [8]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB2

In [15]:
for layer in base_model.layers:
    layer.trainable = False

In [34]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)
x = tf.keras.layers.Dense(7, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

In [1]:
model.summary()

NameError: name 'model' is not defined

In [36]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(
    train_batchs,
    epochs=20,  
    validation_data=validation_batchs,
    callbacks=[reduce_lr,checkpoint]
)

Epoch 1/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 57s 88ms/step - accuracy: 0.6128 - loss: 1.0484 - val_accuracy: 0.5118 - val_loss: 1.3167 - learning_rate: 1.0000e-05
Epoch 2/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 61s 95ms/step - accuracy: 0.6035 - loss: 1.0635 - val_accuracy: 0.5112 - val_loss: 1.3171 - learning_rate: 1.0000e-05
Epoch 3/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 62s 95ms/step - accuracy: 0.6043 - loss: 1.0665 - val_accuracy: 0.5104 - val_loss: 1.3181 - learning_rate: 1.0000e-05
Epoch 4/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 35s 55ms/step - accuracy: 0.6069 - loss: 1.0584 - val_accuracy: 0.5114 - val_loss: 1.3164 - learning_rate: 1.0000e-05
Epoch 5/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 31s 48ms/step - accuracy: 0.6130 - loss: 1.0548 - val_accuracy: 0.5104 - val_loss: 1.3173 - learning_rate: 1.0000e-05
Epoch 6/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.6124 - loss: 1.0579 - val_accuracy: 0.5097 - val_loss: 1.3172 - learning_rate: 1.0000e-05
Epoch 7/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 31s 49ms/ste

In [ ]:
'''
we can continue training the model using this
history = model.fit(
    train_batchs,
    initial_epoch=20,
    epochs=40,  
    validation_data=validation_batchs,
    callbacks=[reduce_lr,checkpoint]
)
'''
